In [81]:
import yfinance as yf
import pandas as pd
import numpy as np
import vectorbt as vbt
import matplotlib.pyplot as plt
import empyrical as ep

# Define the pairs of stocks for each sector
sector_pairs = {
    "Banking": [("BBL.BK", "KBANK.BK"), ("BBL.BK", "KTB.BK"), ("KBANK.BK", "KTB.BK")],
    "Energy_Utilities": [("BGRIM.BK", "GPSC.BK"), ("BGRIM.BK", "GULF.BK"), ("GPSC.BK", "GULF.BK"), ("PTT.BK", "PTTEP.BK")],
    "Finance_Securities": [("MTC.BK", "SAWAD.BK")],
    "Information_Communication_Technology": [("ADVANC.BK", "INTUCH.BK")],
}

# Function to perform backtesting for a pair of stocks
def backtest_pair(pair, data):
    stock1, stock2 = pair

    print(f"Backtesting pair: {stock1}/{stock2}")

    if len(data) < 2:
        raise ValueError(f"Not enough data for {pair}")

    # Calculate the gap between the two stocks
    data["Gap"] = data[stock1] - data[stock2]

    # Calculate the rolling mean and standard deviation of the gap
    window = 30  # Rolling window size
    data["Rolling Mean"] = data["Gap"].rolling(window=window).mean()
    data["Rolling Std"] = data["Gap"].rolling(window=window).std()

    # Calculate the Z-score
    data["Z-score"] = (data["Gap"] - data["Rolling Mean"]) / data["Rolling Std"]

    z_score_buying = -0.5
    z_score_selling = 0.5

    buy_signal = data["Z-score"] < z_score_buying

    sell_signal = data["Z-score"] > z_score_selling

    close_signal = data["Z-score"].abs() < 0.01

    num_buy_orders = buy_signal.sum()
    num_sell_orders = sell_signal.sum()

    try:
        portfolio_buy = vbt.Portfolio.from_signals(
            close=data[stock2],
            entries=buy_signal,
            exits=close_signal,
            init_cash=10000,
            freq="1D",
        )

        portfolio_sell = vbt.Portfolio.from_signals(
            close=data[stock2],
            entries=sell_signal,
            exits=close_signal,
            init_cash=10000,
            freq="1D",
        )

        metrics_buy = {
            "Sharpe Ratio": ep.sharpe_ratio(portfolio_buy.returns()),
            "Sortino Ratio": ep.sortino_ratio(portfolio_buy.returns()),
            "Annual Standard Deviation": ep.annual_volatility(portfolio_buy.returns()),
            "Max Drawdown": ep.max_drawdown(portfolio_buy.returns()),
            "Annual Return": ep.annual_return(portfolio_buy.returns()),
            "Cumulative Return": ep.cum_returns_final(portfolio_buy.returns()),
            "Number of Orders": num_buy_orders,
        }

        metrics_sell = {
            "Sharpe Ratio": ep.sharpe_ratio(portfolio_sell.returns()),
            "Sortino Ratio": ep.sortino_ratio(portfolio_sell.returns()),
            "Annual Standard Deviation": ep.annual_volatility(portfolio_sell.returns()),
            "Max Drawdown": ep.max_drawdown(portfolio_sell.returns()),
            "Annual Return": ep.annual_return(portfolio_sell.returns()),
            "Cumulative Return": ep.cum_returns_final(portfolio_sell.returns()),
            "Number of Orders": num_sell_orders,
        }

        return {
            "Stock Pair": f"{stock1}/{stock2}",
            "Buy Annual Return": metrics_buy["Annual Return"],
            "Buy Cumulative Return": metrics_buy["Cumulative Return"],
            "Buy Annual Std": metrics_buy["Annual Standard Deviation"],
            "Buy Max Drawdown": metrics_buy["Max Drawdown"],
            "Buy Sharpe Ratio": metrics_buy["Sharpe Ratio"],
            "Buy Sortino Ratio": metrics_buy["Sortino Ratio"],
            "Buy Number of Orders": metrics_buy["Number of Orders"],
            "Sell Annual Return": metrics_sell["Annual Return"],
            "Sell Cumulative Return": metrics_sell["Cumulative Return"],
            "Sell Annual Std": metrics_sell["Annual Standard Deviation"],
            "Sell Max Drawdown": metrics_sell["Max Drawdown"],
            "Sell Sharpe Ratio": metrics_sell["Sharpe Ratio"],
            "Sell Sortino Ratio": metrics_sell["Sortino Ratio"],
            "Sell Number of Orders": metrics_sell["Number of Orders"],
        }

    except Exception as e:
        print(f"Error backtesting pair {pair}: {e}")
        return None

results = []
for sector, pairs in sector_pairs.items():
    if pairs:
        print(f"Analyzing sector: {sector}")
        for pair in pairs:
            try:
                data = yf.download(pair, start="2019-07-01", end="2024-07-01")["Adj Close"]
                if not data.empty:
                    result = backtest_pair(pair, data)
                    if result:
                        results.append(result)
                else:
                    print(f"No data found for {pair}")
            except Exception as e:
                print(f"Error fetching data for {pair}: {e}")
                continue
    else:
        print(f"No pairs found for sector: {sector}")

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Display the results DataFrame
print("Backtesting Results:")
print(results_df)

# Calculate average row
avg_row = results_df.select_dtypes(include=np.number).mean()

# Create a DataFrame for the average row
avg_df = pd.DataFrame(avg_row).transpose()

# Rename index to "AVG"
avg_df.index = ["AVG"]

# Concatenate average row with results_df
results_df = pd.concat([results_df, avg_df])

# Save results to Excel
results_df.to_excel("backtesting_results.xlsx")

# Display the updated results DataFrame with average row
print("\nAverage Results:")
results_df


[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[                       0%%                      ]

Analyzing sector: Banking
Backtesting pair: BBL.BK/KBANK.BK
Backtesting pair: BBL.BK/KTB.BK
Backtesting pair: KBANK.BK/KTB.BK
Analyzing sector: Energy_Utilities


[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[                       0%%                      ]

Backtesting pair: BGRIM.BK/GPSC.BK
Backtesting pair: BGRIM.BK/GULF.BK
Backtesting pair: GPSC.BK/GULF.BK
Backtesting pair: PTT.BK/PTTEP.BK
Analyzing sector: Finance_Securities


[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed


Backtesting pair: MTC.BK/SAWAD.BK
Analyzing sector: Information_Communication_Technology
Backtesting pair: ADVANC.BK/INTUCH.BK
Backtesting Results:
            Stock Pair  Buy Annual Return  Buy Cumulative Return  \
0      BBL.BK/KBANK.BK          -0.018103              -0.084048   
1        BBL.BK/KTB.BK           0.059934               0.322752   
2      KBANK.BK/KTB.BK           0.046576               0.244545   
3     BGRIM.BK/GPSC.BK          -0.102143              -0.404153   
4     BGRIM.BK/GULF.BK           0.097676               0.564951   
5      GPSC.BK/GULF.BK           0.087695               0.497742   
6      PTT.BK/PTTEP.BK           0.108793               0.642598   
7      MTC.BK/SAWAD.BK           0.012282               0.060418   
8  ADVANC.BK/INTUCH.BK           0.008547               0.041749   

   Buy Annual Std  Buy Max Drawdown  Buy Sharpe Ratio  Buy Sortino Ratio  \
0        0.315327         -0.584279          0.099049           0.144372   
1        0.264809  

,Stock Pair,Buy Annual Return,Buy Cumulative Return,Buy Annual Std,Buy Max Drawdown,Buy Sharpe Ratio,Buy Sortino Ratio,Buy Number of Orders,Sell Annual Return,Sell Cumulative Return,Sell Annual Std,Sell Max Drawdown,Sell Sharpe Ratio,Sell Sortino Ratio,Sell Number of Orders
0,BBL.BK/KBANK.BK,-0.018103,-0.084048,0.315327,-0.584279,0.099049,0.144372,392.000000,-0.088694,-0.360024,0.314354,-0.629998,-0.138793,-0.199332,477.0
1,BBL.BK/KTB.BK,0.059934,0.322752,0.264809,-0.462215,0.352813,0.513618,486.000000,0.080445,0.450374,0.225822,-0.302763,0.454712,0.715027,459.0
2,KBANK.BK/KTB.BK,0.046576,0.244545,0.268292,-0.481545,0.304403,0.442747,518.000000,0.111417,0.661367,0.232498,-0.315882,0.569711,0.906922,410.0
3,BGRIM.BK/GPSC.BK,-0.102143,-0.404153,0.348487,-0.644615,-0.135389,-0.196273,398.000000,-0.055100,-0.238419,0.364074,-0.561437,0.025222,0.037353,479.0
4,BGRIM.BK/GULF.BK,0.097676,0.564951,0.293101,-0.381464,0.464520,0.690117,521.000000,0.109779,0.649628,0.305210,-0.381464,0.494021,0.729220,431.0
5,GPSC.BK/GULF.BK,0.087695,0.497742,0.305308,-0.381464,0.428059,0.632272,529.000000,0.111760,0.663829,0.310089,-0.381464,0.496767,0.736144,417.0
6,PTT.BK/PTTEP.BK,0.108793,0.642598,0.341855,-0.581277,0.479727,0.662139,557.000000,0.131925,0.813945,0.347891,-0.570321,0.536792,0.742825,338.0
7,MTC.BK/SAWAD.BK,0.012282,0.060418,0.386177,-0.542857,0.224970,0.326102,474.000000,-0.043297,-0.191606,0.402739,-0.542857,0.091755,0.131927,467.0
8,ADVANC.BK/INTUCH.BK,0.008547,0.041749,0.212713,-0.357459,0.146135,0.212971,471.000000,0.027861,0.141171,0.218376,-0.350279,0.234897,0.340725,419.0
AVG,NaN,0.033473,0.209617,0.304008,-0.490797,0.262699,0.380896,482.888889,0.042900,0.287807,0.302339,-0.448496,0.307232,0.460090,433.0
